# 1.Data Import and Combination

In [238]:
import gensim as gs
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import sklearn as sk
import numpy as np
import pickle

## 1.2.import dataset

In [7]:
train_path="DataSet\\train.txt"
test_path="DataSet\\test.txt"
train_df = pd.read_csv(train_path,sep="\t",index_col=0)
test_df=pd.read_csv(test_path,sep="\t",index_col=0)

In [8]:
train_df

,turn1,turn2,turn3,label
id,,,,
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others


## 1.3.get label and turn into category

In [9]:
# set the label into categorical number:
# others:0; happy:1,angry:3;sad:2;

class Label_format:
    def __init__(self,df):
        self.df=df
        
    def get_y(self):
        y = self.df.iloc[:,-1].values
        return y
    
    def label_category(self,y):
        change_list = list()
        emotion = {"others":0,"happy":1,"sad":2,"angry":3}
        for each in y:
            change_list.append(emotion[each])
        change_array=np.array(change_list)
        return change_array
    
    def save_file(self,data,name):
        pickle_file = open('DataSet\\'+name,mode='wb')
        pickle.dump(data,pickle_file)
        pickle_file.close()

In [10]:
#set y label, and we make X later on.

lf = Label_format(train_df)
y = lf.get_y()
y = lf.label_category(y)
lf.save_file(y,"train_y.pkl")

In [11]:
y

array([0, 3, 0, ..., 0, 0, 0])

## 1.4. class for lower text and combination

In [12]:
# because we use pre-trained embeddings, we can't directly use stemming, remove stopwords or remove punctuations.
# we need to turn the data as much as possible close to the pre-trained embedding.
def basic_clean_text(input_text):
    'clean text function'
    #clean emoji, punctions
    input_text = re.sub(r'[^a-zA-Z]',repl=" ",string=input_text)
    #lower the words
    input_text =input_text.lower()
    #stemming the words
    list_text=input_text.split()
    ps=nltk.stem.porter.PorterStemmer()
    list_text_out=list()
    for word in list_text:
        if word in set(stopwords.words('english')):
            continue
        list_text_out.append(ps.stem(word))
    input_text=" ".join(list_text_out) 
    return input_text

def lower_text(input_text):
    #lower the words
    input_text =input_text.lower()
    return input_text
    
def preprocess_method(df):
    turn1_corpus=list()
    turn2_corpus=list()
    turn3_corpus=list()
    #get rows' count
    range_no = train_df.shape[0]
    for row_no in range(0,range_no):
        #clean text by calling function for removing stopwords, stemming and lowering.
        row_text1 = lower_text(df["turn1"][row_no])
        row_text2 = lower_text(df["turn2"][row_no])
        row_text3 = lower_text(df["turn3"][row_no])
        
        turn1_corpus.append(row_text1)
        #print(turn1_corpus)
        turn2_corpus.append(row_text2)
        turn3_corpus.append(row_text3)
    
    # use text_content as dict for storing the data of 
    text_content={"turn1":turn1_corpus,"turn2":turn2_corpus,"turn3":turn3_corpus}
    
    # because we need all the sentences including sentences in turn1, turn2, turn3 
    # to predict the emtion situation, therefore we need to combine them together
    length = len(text_content["turn1"])
    text_all_content = list()
    for i in range(length):
        text_row_all = text_content["turn1"][i] +" " + text_content["turn2"][i] +" "+ text_content["turn3"][i]
        text_all_content.append(text_row_all)
    text_all_content = {"all":text_all_content}
    
    return text_content,text_all_content


In [13]:
text_dict,text = preprocess_method(train_df)

In [14]:
text_dict

{'turn1': ["don't worry  i'm girl",
  'when did i?',
  'by',
  'u r ridiculous',
  'just for time pass',
  "i'm a dog person",
  'so whatsup',
  'ok',
  'really?',
  'bay',
  'i hate my boyfriend',
  'i will do night.',
  'sure go ahead',
  'bad',
  'ok get it......',
  'money money and lots of money😍😍',
  'my gf left ne',
  'get lost',
  'you are lying and i know that',
  'ur creator is very bad',
  'ehat is hehe',
  'do you dance?',
  'i hate it too',
  'not always',
  'bcoz u dont know wat is to miss someone',
  'yeah',
  'i want to tell you something',
  'yes i will send the email and you write all yours filling still i write all matters but your are clever and not any matter for yourself please send andwrite',
  'you are very funny',
  'wth',
  'so rude',
  'ok i will',
  'ntng.....',
  'you are fool',
  'u dare me?',
  'abt me',
  'crazy',
  'goa in india',
  'that was mean',
  'yes :)',
  "i don't no",
  'send me your nekde pic',
  "ok friend's",
  'she is ignoring me',
  'happy

In [15]:
text

{'all': ["don't worry  i'm girl hmm how do i know if you are what's ur name?",
  'when did i? saw many times i think -_- no. i never saw you',
  'by by google chrome where you live',
  'u r ridiculous i might be ridiculous but i am telling the truth. u little disgusting whore',
  'just for time pass wt do u do 4 a living then maybe',
  "i'm a dog person youre so rude whaaaat why",
  'so whatsup nothing much. sitting sipping and watching tv. how abt u? what are you watching on tv?',
  'ok ok im back!! so, how are u',
  'really? really really really really really y saying so many times...i can hear you',
  'bay in the bay 😘 love you',
  'i hate my boyfriend you got a boyfriend? yes',
  'i will do night. alright. keep me in loop. not giving whatsapp no.',
  'sure go ahead many thanks once again! love you too',
  "bad bad bad! that's the bad kind of bad. i have no gf",
  'ok get it...... i made it an option ok',
  "money money and lots of money😍😍 i need to get it tailored but i'm in love w

# 2. check data format 
- for further preprocessing based on pre-trained embedding resource

In [16]:
"""
Created on Tue Oct 13 16:15:11 2020
Embedding Improvement
Because we use pretrained word2vec dataset for embedding, 
so we need to make the result of our preprocessed data close to the format of the pretrained word2vec dataset.

@author: Shenghan ZHANG
"""

'\nCreated on Tue Oct 13 16:15:11 2020\nEmbedding Improvement\nBecause we use pretrained word2vec dataset for embedding, \nso we need to make the result of our preprocessed data close to the format of the pretrained word2vec dataset.\n\n@author: Shenghan ZHANG\n'

In [17]:
import gensim as gs
import pickle
from tqdm import tqdm
import re

## 2.1. Check distinct punctions and emojis

In [18]:
def distinct_punctions(dialogs):
    punc = dict()
    for each_dialog in tqdm(dialogs):
        punc_list = re.findall(r'[^a-zA-Z\d]',each_dialog)
        for each in punc_list:
            try:
                punc[each] +=1
            except KeyError:
                punc[each] = 1
    
    return punc

In [19]:
punc = distinct_punctions(text["all"])

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 181082.18it/s]


In [20]:
punc

{"'": 13424,
 ' ': 379405,
 '?': 14160,
 '-': 834,
 '_': 167,
 '.': 25745,
 '!': 6674,
 ',': 4103,
 '😘': 181,
 '😍': 298,
 '😁': 461,
 '*': 315,
 '😭': 1172,
 '😑': 87,
 ':': 3824,
 ';': 662,
 '^': 283,
 ')': 2123,
 '😻': 87,
 '😂': 3155,
 '😢': 477,
 '👍': 98,
 '😀': 253,
 '"': 520,
 '🙂': 131,
 '<': 187,
 '😓': 11,
 '😒': 166,
 '😈': 11,
 '👿': 12,
 '🖑': 1,
 '😾': 57,
 '😠': 162,
 '(': 929,
 '👻': 5,
 '=': 159,
 '‑': 143,
 '💔': 165,
 '😥': 55,
 '😞': 492,
 '😪': 25,
 '😤': 131,
 '😃': 204,
 '😦': 48,
 '😼': 4,
 '😏': 67,
 '#': 44,
 '😱': 19,
 '&': 239,
 '😬': 62,
 '🙁': 62,
 '/': 370,
 '😺': 108,
 '😣': 28,
 '’': 349,
 '%': 40,
 '😆': 242,
 '😄': 214,
 '😅': 235,
 '̇': 20,
 '�': 9,
 '😊': 188,
 '😕': 29,
 '😽': 79,
 '・': 26,
 '🙀': 47,
 '🤣': 32,
 '🤐': 5,
 '😡': 377,
 '👌': 25,
 '¿': 18,
 '😮': 12,
 '❤': 145,
 '️': 147,
 '👼': 3,
 '🏻': 32,
 '~': 188,
 '🙄': 42,
 '\\': 109,
 '😐': 38,
 '😿': 43,
 '☹': 54,
 '😉': 113,
 'ı': 14,
 '😋': 97,
 '😹': 117,
 'é': 4,
 '🚗': 2,
 '🚘': 1,
 '😜': 110,
 '😴': 11,
 '💤': 6,
 '😇': 29,
 '😔': 75,
 '>': 

## 2.2. check distinct vocabulary

In [346]:
def distinct_words(dialogs):
    vocab = dict()
    for each_dialog in tqdm(dialogs):
        words_list = each_dialog.split(sep=" ")
        for word in words_list:
            if word == '':
                continue
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    
    return vocab

In [171]:
vocab_ori = distinct_words(text["all"])
print("the length of distinct vocabulary is:%d"%len(vocab_ori))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 252017.05it/s]

the length of distinct vocabulary is:16658


In [172]:
vocab_ori

{'': 57465,
 'don': 3260,
 "'": 13182,
 't': 7988,
 'worry': 64,
 'i': 24590,
 'm': 4567,
 'girl': 450,
 'hmm': 277,
 'how': 2563,
 'do': 3366,
 'know': 2662,
 'if': 965,
 'you': 20067,
 'are': 5871,
 'what': 6330,
 's': 5371,
 'ur': 943,
 'name': 432,
 '?': 10453,
 'when': 632,
 'did': 841,
 'saw': 66,
 'many': 226,
 'times': 99,
 'think': 891,
 '-_-': 25,
 'n': 1590,
 'o': 1502,
 '.': 11436,
 'never': 531,
 'by': 481,
 'google': 80,
 'chrome': 4,
 'where': 888,
 'live': 172,
 'u': 8180,
 'r': 2121,
 'ridiculous': 5,
 'might': 77,
 'be': 1747,
 'but': 2203,
 'am': 3662,
 'telling': 88,
 'the': 4961,
 'trut': 17,
 'h': 744,
 'little': 156,
 'disgusting': 19,
 'whore': 7,
 'just': 2075,
 'for': 2533,
 'time': 725,
 'pass': 23,
 'wt': 84,
 '4': 94,
 'a': 6019,
 'living': 43,
 'then': 1141,
 'maybe': 140,
 'dog': 51,
 'person': 221,
 'youre': 51,
 'so': 3748,
 'rude': 412,
 'whaaaat': 1,
 'why': 2591,
 'whatsup': 7,
 'nothing': 716,
 'muc': 79,
 'sitting': 28,
 'sipping': 1,
 'and': 3117,

In [352]:
vocab1 = distinct_words(cleaned_text_all["all"])
print("the length of distinct vocabulary is:%d"%len(vocab1))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 249802.94it/s]

the length of distinct vocabulary is:16657


In [353]:
vocab1

{'don': 3260,
 "'": 13182,
 't': 7988,
 'worry': 64,
 'i': 24590,
 'm': 4567,
 'girl': 450,
 'hmm': 277,
 'how': 2563,
 'do': 3366,
 'know': 2662,
 'if': 965,
 'you': 20067,
 'are': 5871,
 'what': 6330,
 's': 5371,
 'ur': 943,
 'name': 432,
 '?': 10453,
 'when': 632,
 'did': 841,
 'saw': 66,
 'many': 226,
 'times': 99,
 'think': 891,
 '-_-': 25,
 'n': 1590,
 'o': 1502,
 '.': 11436,
 'never': 531,
 'by': 481,
 'google': 80,
 'chrome': 4,
 'where': 888,
 'live': 172,
 'u': 8180,
 'r': 2121,
 'ridiculous': 5,
 'might': 77,
 'be': 1747,
 'but': 2203,
 'am': 3662,
 'telling': 88,
 'the': 4961,
 'trut': 17,
 'h': 744,
 'little': 156,
 'disgusting': 19,
 'whore': 7,
 'just': 2075,
 'for': 2533,
 'time': 725,
 'pass': 23,
 'wt': 84,
 '4': 94,
 'a': 6019,
 'living': 43,
 'then': 1141,
 'maybe': 140,
 'dog': 51,
 'person': 221,
 'youre': 51,
 'so': 3748,
 'rude': 412,
 'whaaaat': 1,
 'why': 2591,
 'whatsup': 7,
 'nothing': 716,
 'muc': 79,
 'sitting': 28,
 'sipping': 1,
 'and': 3117,
 'watching'

In [354]:
vocab2 = distinct_words(cleaned_text_all2["all"])
print("the length of distinct vocabulary is:%d"%len(vocab2))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 262959.76it/s]

the length of distinct vocabulary is:16654


In [355]:
vocab3 = distinct_words(cleaned_text_all3["all"])
print("the length of distinct vocabulary is:%d"%len(vocab3))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 260776.24it/s]

the length of distinct vocabulary is:16277


In [356]:
vocab4 = distinct_words(cleaned_text_all4["all"])
print("the length of distinct vocabulary is:%d"%len(vocab4))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 279811.38it/s]

the length of distinct vocabulary is:15199


In [357]:
vocab5 = distinct_words(cleaned_text_all5["all"])
print("the length of distinct vocabulary is:%d"%len(vocab5))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 305455.72it/s]

the length of distinct vocabulary is:15129


In [358]:
vocab6 = distinct_words(cleaned_text_all6["all"])
print("the length of distinct vocabulary is:%d"%len(vocab6))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 332180.22it/s]

the length of distinct vocabulary is:15125


In [359]:
vocab7 = distinct_words(cleaned_text_all7["all"])
print("the length of distinct vocabulary is:%d"%len(vocab7))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 315004.68it/s]

the length of distinct vocabulary is:11264


In [360]:
vocab8 = distinct_words(cleaned_text_all8["all"])
print("the length of distinct vocabulary is:%d"%len(vocab8))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 347449.77it/s]

the length of distinct vocabulary is:11260


In [361]:
vocab9 = distinct_words(cleaned_text_all9["all"])
print("the length of distinct vocabulary is:%d"%len(vocab9))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 315007.82it/s]

the length of distinct vocabulary is:11231


## 2.3. check the percentage of words in data can be processed by word2vec pre-trained data

### 2.3.1. Data with Emoji, Emoticon and Punctuations

In [24]:
# import embedding resource
import gensim as gs
pretrained_word2vec_path = "DataSet\\GoogleNews-vectors-negative300.bin"
word2vec_pre = gs.models.KeyedVectors.load_word2vec_format(pretrained_word2vec_path, binary= True)

In [46]:
# build the function for check the coverage of the vocabulary and the text
import operator
def check_coverage(vocab, embedding_resource):
    cov_vocab =0
    num_vocab = len(vocab)
    cov_text = 0
    not_cov = dict()
    not_text = 0
    for word in tqdm(vocab):
        try:
            x = embedding_resource[word]
            cov_vocab += 1
            cov_text += vocab[word]
        except:
            not_cov[word]=vocab[word]
            not_text += vocab[word]
            pass
    percent_cov_vocab = cov_vocab/num_vocab
    percent_cov_text = cov_text/(cov_text+not_text)
    print("In Embedding Index we have {:.2%} coverage of distinct vocabulary".format(percent_cov_vocab))
    print("And we have {:.2%} coverage of all text".format(percent_cov_text))
    sorted_not_cov = sorted(not_cov.items(),key= operator.itemgetter(1),reverse = True)
    return sorted_not_cov

In [362]:
not_cov = check_coverage(vocab_ori, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 16658/16658 [00:00<00:00, 265123.21it/s]

In Embedding Index we have 65.78% coverage of distinct vocabulary
And we have 72.44% coverage of all text


In [363]:
not_cov[:20]# so here we can see in google trained embedding index, we can use emoji, punctuation and emoticons, and those stopwords
# and we also need to take care of words abreviation such as "i'm" and take care of the words sticked with punctuations

[('', 57465),
 ("'", 13182),
 ('.', 11436),
 ('?', 10453),
 ('to', 8131),
 ('a', 6019),
 (',', 4103),
 ('!', 3480),
 ('😂', 3155),
 ('and', 3117),
 ('of', 2575),
 ('...', 1948),
 ('..', 1815),
 (':', 1723),
 ('😭', 1172),
 ('(', 929),
 ('??', 866),
 ('!!', 599),
 ('😞', 492),
 ('😢', 477)]

In [364]:
# after assign space to each word and seperate them correctly
not_cov1 = check_coverage(vocab1, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 16657/16657 [00:00<00:00, 397639.82it/s]

In Embedding Index we have 65.79% coverage of distinct vocabulary
And we have 80.74% coverage of all text


In [365]:
# after correct common words
not_cov2 = check_coverage(vocab2, word2vec_pre) # the result of coverage of all text is better

100%|████████████████████████████████████████████████████████████████████████| 16654/16654 [00:00<00:00, 506012.13it/s]

In Embedding Index we have 65.78% coverage of distinct vocabulary
And we have 82.02% coverage of all text


In [366]:
# after correct missing charactors
not_cov3 = check_coverage(vocab3, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 16277/16277 [00:00<00:00, 466291.60it/s]

In Embedding Index we have 66.03% coverage of distinct vocabulary
And we have 81.59% coverage of all text


In [367]:
not_cov3[:20] # here we can see, it's mostly due to punctuatuations, stopwords and emoji, emoticons

[('.', 11436),
 ('?', 10453),
 ('to', 7896),
 ('a', 5975),
 ("'", 5319),
 (',', 4103),
 ('!', 3480),
 ('😂', 3155),
 ('and', 3119),
 ('of', 2553),
 ('...', 1948),
 ('..', 1815),
 (':', 1723),
 ('😭', 1172),
 ('(', 929),
 ('??', 866),
 ('!!', 599),
 ('😞', 492),
 ('😢', 477),
 ('😁', 461)]

In [368]:
# after extract emoji and emoticons
not_cov4 = check_coverage(vocab4, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15199/15199 [00:00<00:00, 346363.42it/s]

In Embedding Index we have 70.31% coverage of distinct vocabulary
And we have 85.63% coverage of all text


In [369]:
not_cov4[:20] # now here we can see the problems aside with stopwords, punctuations and incorrect spell of words

[('.', 11436),
 ('?', 10453),
 ('to', 7896),
 ('a', 5975),
 ("'", 5319),
 (',', 4103),
 ('!', 3480),
 ('and', 3119),
 ('of', 2553),
 (':', 1723),
 ('"', 349),
 (';', 283),
 ('-', 263),
 (')', 115),
 ('/', 112),
 ('funn', 85),
 ('favourite', 79),
 ('didnt', 78),
 ('sorr', 74),
 ('howr', 56)]

In [370]:
# after remove punctuations
not_cov5 = check_coverage(vocab5, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15129/15129 [00:00<00:00, 433330.55it/s]

In Embedding Index we have 70.41% coverage of distinct vocabulary
And we have 93.46% coverage of all text


In [371]:
not_cov5[:40] # we can use this as reference to determine which words should be assigned to stopwords list.

[('to', 7896),
 ('a', 5975),
 ('and', 3119),
 ('of', 2553),
 ('funn', 85),
 ('favourite', 79),
 ('didnt', 78),
 ('sorr', 74),
 ('howr', 56),
 ('ohk', 50),
 ('whatsapp', 49),
 ('yess', 49),
 ('whatr', 42),
 ('angr', 37),
 ('whyr', 33),
 ('doesnt', 32),
 ('okk', 31),
 ('wherer', 31),
 ('plzz', 30),
 ('bcz', 30),
 ('humour', 28),
 ('colour', 26),
 ('seriousl', 23),
 ('exactl', 22),
 ('whor', 22),
 ('okkk', 22),
 ('frnds', 21),
 ('chating', 21),
 ('budd', 20),
 ('anywa', 18),
 ('nthng', 18),
 ('lonel', 17),
 ('selfie', 17),
 ('ranbir', 17),
 ('wbu', 16),
 ('isnt', 15),
 ('yrr', 15),
 ('willd', 15),
 ('craz', 15),
 ('programme', 14)]

In [372]:
# after remove stopwords
not_cov6 = check_coverage(vocab6, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15125/15125 [00:00<00:00, 421260.27it/s]

In Embedding Index we have 70.43% coverage of distinct vocabulary
And we have 98.13% coverage of all text


In [373]:
not_cov6[:30] # now we have the only problem is about incorrect spell，abreviation，shorthand words.
# lets firstly use pyspellchecker to correct the incorrect spell.

[('funn', 85),
 ('favourite', 79),
 ('didnt', 78),
 ('sorr', 74),
 ('howr', 56),
 ('ohk', 50),
 ('whatsapp', 49),
 ('yess', 49),
 ('whatr', 42),
 ('angr', 37),
 ('whyr', 33),
 ('doesnt', 32),
 ('okk', 31),
 ('wherer', 31),
 ('plzz', 30),
 ('bcz', 30),
 ('humour', 28),
 ('colour', 26),
 ('seriousl', 23),
 ('exactl', 22),
 ('whor', 22),
 ('okkk', 22),
 ('frnds', 21),
 ('chating', 21),
 ('budd', 20),
 ('anywa', 18),
 ('nthng', 18),
 ('lonel', 17),
 ('selfie', 17),
 ('ranbir', 17)]

In [374]:
# after correction of incorrect spell of words
not_cov7 = check_coverage(vocab7, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11264/11264 [00:00<00:00, 434369.57it/s]

In Embedding Index we have 88.79% coverage of distinct vocabulary
And we have 99.32% coverage of all text


In [375]:
not_cov7[:10]

[('of', 241),
 ('goole', 90),
 ('favourite', 81),
 ('to', 60),
 ('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('emosi', 40),
 ('a', 40),
 ('dumas', 33)]

In [376]:
# after correction of incorrect spell of words, we need to remove stopwords again
not_cov8 = check_coverage(vocab8, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11260/11260 [00:00<00:00, 470097.38it/s]

In Embedding Index we have 88.82% coverage of distinct vocabulary
And we have 99.41% coverage of all text


In [380]:
not_cov8[:30]

[('goole', 90),
 ('favourite', 81),
 ('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('emosi', 40),
 ('dumas', 33),
 ('he/she', 31),
 ('humour', 29),
 ('colour', 27),
 ('budd', 20),
 ('ranbir', 19),
 ('programme', 15),
 ('travelling', 13),
 ('aaaaa', 11),
 ('shut-up', 11),
 ('trw', 11),
 ('oho', 10),
 ('fyn', 10),
 ('favourites', 9),
 ('learnt', 9),
 ('wasnt', 9),
 ('friendzone', 8),
 ('bessie', 8),
 ('ha-ha', 8),
 ('cortina', 8),
 ('helena', 8),
 ('pyy', 8),
 ('natasha', 8),
 ('naaah', 7)]

In [378]:
# correct the common shorthand writing and some synonym
not_cov9 = check_coverage(vocab9, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11231/11231 [00:00<00:00, 489588.30it/s]

In Embedding Index we have 89.07% coverage of distinct vocabulary
And we have 99.54% coverage of all text


In [379]:
not_cov9

[('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('dumas', 33),
 ('ranbir', 19),
 ('trw', 11),
 ('fyn', 10),
 ('bessie', 8),
 ('cortina', 8),
 ('helena', 8),
 ('pyy', 8),
 ('natasha', 8),
 ('grey', 7),
 ('bahubali', 7),
 ('sheehan', 7),
 ('tajmahal', 7),
 ('wowwww', 6),
 ('alida', 6),
 ('sinfield', 6),
 ('malaya', 6),
 ('krk', 5),
 ('jarvis', 5),
 ('gujarati', 5),
 ('raoul', 5),
 ('usl', 5),
 ('samoa', 5),
 ('computerise', 5),
 ('wcm', 4),
 ('deepika', 4),
 ('mohanlal', 4),
 ('whatsaap', 4),
 ('karol', 4),
 ('didcot', 4),
 ('mrna', 4),
 ('suu', 4),
 ('heheheh', 4),
 ('lollo', 4),
 ('mumby', 4),
 ('kalkara', 4),
 ('urr', 4),
 ('awsd', 4),
 ('kingman', 4),
 ('haag', 4),
 ('margi', 4),
 ('hera', 4),
 ('laz', 4),
 ('sabah', 4),
 ('nps', 4),
 ('signalling', 3),
 ('lolxxx', 3),
 ('lolzzz', 3),
 ('uuuc', 3),
 ('poona', 3),
 ('kenna', 3),
 ('useropenreflink', 3),
 ('wordstar', 3),
 ('uttarakhand', 3),
 ('sunda', 3),
 ('bhabha', 3),
 ('yayyyy', 3),
 ('golmaal', 3),
 ('atb', 3),
 ('django', 3),


# 3.split emoji, emoticons and punctuations
 - e.g. "you?" ? and you are adhered together, which needs to be split
 - e.g. "yess i am crazyyy😂😂" I also need to split emoji from the word "crazyyy"

## 3.1. Test

In [191]:
# test for getting all emoji based on list above
input_text="😭😭 :)a fsdfsdf😀 sdf?-#%$&_👍 .....~~!:"
emoji_list = re.findall(r'(?:[^ a-zA-Z\d\/\*\:\)\.\,\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
#emoji_list = re.sub(r'(?:[^a-zA-Z:\)\.\?\^\;])',r' \g<0> ',input_text).strip()
print(emoji_list)

['😭', '😭', '😀', '👍']


In [189]:
# test for getting all emoticons based on list above
input_text = "*_* ;) :) </3 :-) 😭 sdfsdf^_^adfs? sfd? ?"
emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
print(emoticon_list)

['*_*', ';)', ':)', '</3', ':-)', '^_^']


In [53]:
# test for only getting punctuations based on the list above
input_text = "how r u!? i don't kno, btw??? !? |!!!??? ...."
# actually we extract punctuations by (\!+|\?+|\.+), but have to make sure it's not part of emoticons
punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!+|\?+|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
print(punctuation_list)

['???']


In [54]:
# test for seperate the sentence from sticked words combination into split one
# I make each word with space split from punctuations.
input_text = "yess i am crazyyy😂😂 then stop laughing 😂 hehe...u also laugh u stop laughing dude"
sentence_split = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
sentence_split = re.sub(r'  +',r' ',sentence_split).strip()
print(sentence_split)

yess i am crazyyy 😂😂 then stop laughing 😂 hehe ... u also laugh u stop laughing dude


## 3.2. seperate the adhered terms

In [192]:
# functions for extracting emoji, emoticons and punctuations
def extract_emoji(input_text):
    emoji_list = re.findall(r'(?:[^a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
    return emoji_list
    
def extract_emoticon(input_text):
    emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
    return emoticon_list

def extract_punctuation(input_text):
    'here I use for loop to split the double punctuations into single ones'
    punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
    result_list=list()
    for each in punctuation_list:
        list_split=[each]
        if each !="...":
            list_split=list(each)
        for punc in list_split:
            result_list.append(punc)
    return result_list

# functions for assigning emoji, emoticons and punctuations space from each other
def assign_space(input_text):
    #emoji
    input_text = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',' \g<0> ',input_text)
    #emoticon
    input_text = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',' \g<0> ',input_text)
    #punctuation
    input_text = re.sub(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',' \g<0> ' ,input_text)
    #replace the redundant space
    input_text = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
    input_text = re.sub(r'  +',r' ',input_text)
    return input_text

In [107]:
def get_cleaned_text(input_dict):
    for each_key in input_dict.keys():
        cleaned_sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            cleaned_sentence = assign_space(sentence)
            cleaned_sentences.append(cleaned_sentence)
        input_dict[each_key]=cleaned_sentences
    return input_dict

cleaned_text = get_cleaned_text(text_dict)
cleaned_text_all = get_cleaned_text(text)

    

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:01<00:00, 17560.97it/s]


In [108]:
cleaned_text_all

{'all': [" don ' t worry i ' m girl hmm how do i know if you are what ' s ur name ?",
  ' when did i ? saw many times i think -_- n o . i never saw you ',
  ' by by google chrome where you live ',
  ' u r ridiculous i might be ridiculous but i am telling the trut h . u little disgusting whore ',
  ' just for time pass wt do u do 4 a living then maybe ',
  " i ' m a dog person youre so rude whaaaat why ",
  ' so whatsup nothing muc h . sitting sipping and watching t v . how abt u ? what are you watching on tv ?',
  ' ok ok im back !! so , how are u ',
  ' reall y ? really really really really really y saying so many times ... i can hear you ',
  ' bay in the bay 😘 love you ',
  ' i hate my boyfriend you got a boyfrien d ? yes ',
  ' i will do nigh t . alrigh t . keep me in loo p . not giving whatsapp no .',
  ' sure go ahead many thanks once agai n ! love you too ',
  " bad bad ba d ! that ' s the bad kind of ba d . i have no gf ",
  ' ok get it ...... i made it an option ok ',
  " mone

# 4. Correct the words in sentence
## 4.1. Correct common words

In [162]:
# turn don't, doesn't, can't, couldn't, shouldn't, didn't, isn't, wasn't, weren't, aren't, won't, shan't, 's, 're
# as do not, do not, can not, could not, should not, did not, is not, was not, were not, are not, will not, shall not, 's, are

# turn r , u, bt, n, 
# as are, you ,but, and
cor_dict= {
    "don ' t":      "do not",
    "doesn ' t":    "do not",
    "can ' t":      "can not",
    "couldn ' t":   "could not",
    "shouldn ' t":  "should not",
    "didn ' t":     "did not",
    "isn ' t":      "is not",
    "wasn ' t":     "was not",
    "weren ' t":    "were not",
    "aren ' t":     "are not",
    "won ' t":      "will not",
    "shan ' t":     "shall not",
    
    #"\' re ":       "are ",
    #" re ":         "are",
    "\' m ":        "am ",
    "\' ll ":       "will ",
    #" \' s ":       " \'s "

    }

In [163]:
def correct_words_common(input_dict):
    'for correcting the words in sentences based on hand-crafted data'
    output_dict = dict()
    for each_key_1 in input_dict.keys():
        corrected_sentences = list()
        for sentence in tqdm(input_dict[each_key_1]):
            corrected_sentence = sentence
            for each_key_2 in cor_dict.keys():
                if each_key_2 in sentence:
                    corrected_sentence = corrected_sentence.replace(each_key_2,cor_dict[each_key_2])
            corrected_sentences.append(corrected_sentence)
        output_dict[each_key_1]=corrected_sentences
    return output_dict

cleaned_text2 = correct_words_common(cleaned_text)
cleaned_text_all2 = correct_words_common(cleaned_text_all)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 472308.25it/s]


In [164]:
vocab2 = distinct_words(cleaned_text_all2["all"])
print("the length of distinct vocabulary is:%d"%len(vocab2))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 272438.35it/s]

the length of distinct vocabulary is:16655


## 4.2. Correct missing characters
- e.g. "hmmm i can talk no w", here "now" is missing "w".

In [167]:
def correct_words_combin(input_dict):
    output_dict = dict()
    for each_key in input_dict.keys():
        corrected_sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            corrected_sentence = sentence
            corrected_sentence = re.sub(r' ([a-z]+) ([bcdefghjklnopqrstvwxz]) ',r' \g<1>\g<2> ',corrected_sentence)
            corrected_sentences.append(corrected_sentence)
        output_dict[each_key]=corrected_sentences
    return output_dict

ww = {"turn1":["I ' m go d , and I wasn ' t player wai t ", "hhh you ' re kidding m e"]}
cleaned_text3 = correct_words_combin(cleaned_text2)
cleaned_text_all3 = correct_words_combin(cleaned_text_all2)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 210006.26it/s]


In [168]:
vocab3 = distinct_words(cleaned_text_all3["all"])
print("the length of distinct vocabulary is:%d"%len(vocab3))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 272350.95it/s]

the length of distinct vocabulary is:16278


In [199]:
cleaned_text_all3["all"][:10]

[" do not worry i am girl hmm how do i know if you are what ' s ur name ?",
 ' when did i ? saw many times i think -_- no . i never saw you ',
 ' by by google chrome where you live ',
 ' ur ridiculous i might be ridiculous but i am telling the truth . u little disgusting whore ',
 ' just for time pass wt do u do 4 a living then maybe ',
 ' i am a dog person youre so rude whaaaat why ',
 ' so whatsup nothing much . sitting sipping and watchingt v . how abt u ? what are you watching on tv ?',
 ' ok ok im back !! so , how are u ',
 ' reall y ? really really really really really y saying so many times ... i can hear you ',
 ' bay in the bay 😘 love you ']

# 5. Further Cleaning
## 5.1. Remove emoji, emoticons
- but in order to use emoji and emoticons later, we need to save them for later use

In [193]:
# getting all emoji, emoticons and save them in a new dictionary
def extract_emo(input_dict):
    output_dict = dict()
    emo_save=dict()
    for each_key in input_dict.keys():
        emo_dict = dict()
        emo_dict["emoji"]=list()
        emo_dict["emoticon"]=list()
        sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            new_sentence = sentence
            emoji_list = re.findall(r'(?:[^ a-zA-Z\d\/\*\:\)\.\,\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',sentence)
            emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',sentence)
            emo_dict["emoji"].append(emoji_list)
            emo_dict["emoticon"].append(emoticon_list)
            new_sentence = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',"",new_sentence)
            new_sentence = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',"",new_sentence)
            sentences.append(new_sentence)
        emo_save[each_key]=emo_dict
        output_dict[each_key]=sentences
    
    return emo_save,output_dict

In [220]:
emo_save, cleaned_text4 = extract_emo(cleaned_text3)
emo_save_all, cleaned_text_all4 = extract_emo(cleaned_text_all3)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 147542.41it/s]


In [226]:
cleaned_text_all4["all"][:10]

[" do not worry i am girl hmm how do i know if you are what ' s ur name ?",
 ' when did i ? saw many times i think  no . i never saw you ',
 ' by by google chrome where you live ',
 ' ur ridiculous i might be ridiculous but i am telling the truth . u little disgusting whore ',
 ' just for time pass wt do u do 4 a living then maybe ',
 ' i am a dog person youre so rude whaaaat why ',
 ' so whatsup nothing much . sitting sipping and watchingt v . how abt u ? what are you watching on tv ?',
 ' ok ok im back  so , how are u ',
 ' reall y ? really really really really really y saying so many times  i can hear you ',
 ' bay in the bay  love you ']

In [210]:
emo_save_all["all"]["emoji"][:15]

[[], [], [], [], [], [], [], [], [], ['😘'], [], [], [], [], []]

In [212]:
emo_save_all["all"]["emoticon"][:13]

[[], ['-_-'], [], [], [], [], [], ['!!'], ['...'], [], [], [], []]

## 5.2. Remove punctuations

In [230]:
def remove_punc(input_dict):
    output_dict = dict()
    for each_key in input_dict.keys():
        sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            new_sentence = sentence
            new_sentence = re.sub(r'[^ a-zA-Z]',"",new_sentence).strip()
            sentences.append(new_sentence)
        output_dict[each_key]=sentences
    return output_dict

In [231]:
cleaned_text5 = remove_punc(cleaned_text4)
cleaned_text_all5 = remove_punc(cleaned_text_all4)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 560025.36it/s]


In [232]:
cleaned_text_all5["all"][:10]

['do not worry i am girl hmm how do i know if you are what  s ur name',
 'when did i  saw many times i think  no  i never saw you',
 'by by google chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth  u little disgusting whore',
 'just for time pass wt do u do  a living then maybe',
 'i am a dog person youre so rude whaaaat why',
 'so whatsup nothing much  sitting sipping and watchingt v  how abt u  what are you watching on tv',
 'ok ok im back  so  how are u',
 'reall y  really really really really really y saying so many times  i can hear you',
 'bay in the bay  love you']

## 5.3.  Remove stopwords made by handcrafted

In [243]:
print(stopwords.words('english')) # which is not suitable here, coz e.g. in emotion analysis, not happy is opposite from happy.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [245]:
stopwords_byhand = ["to","a","and","of"]

In [246]:
def remove_stopwords(input_dict):
    output_dict = dict()
    for each_key in input_dict.keys():
        sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            #new_sentence = sentence
            words = sentence.split()
            new_words=list()
            for word in words:
                if word in stopwords_byhand:
                    continue
                else:
                    new_words.append(word)
            sentence = " ".join(new_words)
            sentences.append(sentence)
        output_dict[each_key]=sentences
    return output_dict

In [247]:
cleaned_text6 = remove_stopwords(cleaned_text5)
cleaned_text_all6 = remove_stopwords(cleaned_text_all5)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 332383.59it/s]


In [268]:
# after spell mistakce correction, it's needed to run over again to remove the stopwords
cleaned_text8 = remove_stopwords(cleaned_text7)
cleaned_text_all8 = remove_stopwords(cleaned_text_all7)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 351643.43it/s]


# 6. Correction of Incorrect Spell and Shorthand Words
## 6.1. Correcting spell mistake
 - e.g. "nt" is actually "not", "noo" is actually "no"
 - in order to do so, here i use library called pyspellchecker

In [253]:
from spellchecker import SpellChecker
def correct_spell(input_dict):
    output_dict = dict()
    spell = SpellChecker()
    for each_key in input_dict.keys():
        sentences = list()
        for sentence in tqdm(input_dict[each_key]):
            #new_sentence = sentence
            words = sentence.split()
            new_words=list()
            for word in words:
                word = spell.correction(word)
                new_words.append(word)
            sentence = " ".join(new_words)
            sentences.append(sentence)
        output_dict[each_key]=sentences
    return output_dict


In [259]:
cleaned_text7 = correct_spell(cleaned_text6)
cleaned_text_all7 = correct_spell(cleaned_text_all6)

100%|████████████████████████████████████████████████████████████████████████████| 30160/30160 [24:21<00:00, 20.63it/s]


In [260]:
cleaned_text_all7["all"][:10]

['do not worry i am girl hmm how do i know if you are what s ur name',
 'when did i saw many times i think no i never saw you',
 'by by goole chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth u little disgusting whore',
 'just for time pass wt do u do living then maybe',
 'i am dog person your so rude whaaaat why',
 'so whatsit nothing much sitting sipping watching v how at u what are you watching on tv',
 'ok ok im back so how are u',
 'really y really really really really really y saying so many times i can hear you',
 'bay in the bay love you']

## 6.2. shorthand words correction
- here we only deal with the most frequent words happened in text which can't be changed into vectors based on pretrained word2vec index

In [389]:
cor_dict2= {
    "goole" : "google",
    "favourite":"favorite",
    "he/she":"he or she",
    "ur":"your",
    "u":"you",
    "s":"is",
    "humour":"humor",
    "budd":"buddy",
    "colour":"color",
    "emosi":"emoji",
    "programme":"program",
    "travelling":"travel",
    "aaaaa":"ah",
    "shut-up":"shut up",
    "favourites":"favorite",
    "oho":"oh",
    "learnt":"learn",
    "wasnt":"was not",
    "ha-ha":"ha ha",
    "naaah":"no",
    "moodie":"moody",
    "ah-ha":"ah ha",
    "wowww":"wow",
    "okkkkk":"ok",
    "hahahha":"ha ha ha",
    "frida":"friday",
    "friendzone":"friend zone",
    "friendzoned":"friend zone",
    "realise":"realize",
    "computerised":"computerise",
    "apologise":"apologize",
    "youuuu":"you",
    "starbuck":"starbucks",
    "realised":"realize",
    "judgement":"judge",
    "honoured":"honor",
    "organisation":"organization",
    "ddl":"deadline",
    }

In [390]:
def correct_words_common2(input_dict):
    'for correcting the words in sentences based on hand-crafted data'
    output_dict = dict()
    for each_key_1 in input_dict.keys():
        corrected_sentences = list()
        for sentence in tqdm(input_dict[each_key_1]):
            corrected_sentence = " "+sentence+" "
            for each_key_2 in cor_dict2.keys():
                if each_key_2 in sentence:
                    rule = " "+each_key_2+" "
                    corrected_sentence = re.sub(rule," "+cor_dict2[each_key_2]+" ",corrected_sentence)
                    #corrected_sentence.replace(each_key_2,cor_dict2[each_key_2])
            corrected_sentences.append(corrected_sentence)
        output_dict[each_key_1]=corrected_sentences
    return output_dict

cleaned_text9 = correct_words_common2(cleaned_text8)
cleaned_text_all9 = correct_words_common2(cleaned_text_all8)

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 167076.16it/s]


In [391]:
vocab9 = distinct_words(cleaned_text_all9["all"])
print("the length of distinct vocabulary is:%d"%len(vocab9))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 325167.43it/s]

the length of distinct vocabulary is:11230


In [392]:
# correct the common shorthand writing and some synonym
not_cov9 = check_coverage(vocab9, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11230/11230 [00:00<00:00, 417593.44it/s]

In Embedding Index we have 89.07% coverage of distinct vocabulary
And we have 99.54% coverage of all text


# 7. data storage

In [393]:
# finally we have 2 versions of preprocessing of data so far
# first one is preprocessed train data without spelling and shorthand words correction, 
# but after assigning space, correction of common shorthand words, adding missing charactors, removal of stopwords, emoji, emoticon and punctuations
# the second one is preprocessed train data with spelling and shorthand words correction.
# and I also have extracted all the emoji and emoticons of the data, which can be later processed for further analysis

# here I gather all coding in other 2 jpynb file for pure preprocessing without checking data format in Preprocessing

In [394]:
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [395]:
save_data("pre_train_3turn_no_correct.pkl",cleaned_text6)
save_data("pre_train_all_no_correct.pkl",cleaned_text_all6)
save_data("pre_train_3turn_no_emoji.pkl",cleaned_text9)
save_data("pre_train_all_no_emoji.pkl",cleaned_text_all9)
save_data("train_emoji_emoti_3turn.pkl",emo_save)
save_data("train_emoji_emoti_all.pkl",emo_save_all)